In [ ]:
import httpx
import os
import json

from src.download.weatherdata_downloader import WeatherdataDownloader
from src.download.patients import PatientsDownload
from src.utils.graph_configurations import get_weatherdata_timeline_plot
import pandas as pd

### Use Case: Von einer Datei alle Patient:innen extrahieren und für einzelne Patient:innen Wetterdaten anschauen - Tagesmittelwerte

In [ ]:
# INPUT from the User Interface 
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind Daten ab dem 01.01.2010 verfügbar: ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD): ")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Weather Data
weather_data = WeatherdataDownloader()
daily_means = weather_data.get_weatherdata_patient(latitude = interesting_patient.address.latitude, longitude = interesting_patient.address.longitude, start_date = START_DATE, end_date = END_DATE)

In [ ]:
daily_means.head()

In [ ]:
get_weatherdata_timeline_plot(data = daily_means, column = 'Niederschlag (mm)')

In [ ]:
get_weatherdata_timeline_plot(data = daily_means, column = 'Sonnenscheindauer (min)')

### Use Case: Von einer Datei alle Patient:innen extrahieren und für einzelne Patient:innen Wetterdaten anschauen - Durchschnitt/ Median in einem bestimmten Zeitraum

In [ ]:
# INPUT from the User Interface 
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind Daten ab dem 01.01.2010 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD): ")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_small.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data + select an interesting patient
patients = PatientsDownload()
patients.extract_patients(data = data)
interesting_patient = patients.get_patient_by_id("595")

In [ ]:
# Get Weather Data
weather_data = WeatherdataDownloader()
daily_means = weather_data.get_weatherdata_patient(latitude = interesting_patient.address.latitude, longitude = interesting_patient.address.longitude, start_date = START_DATE, end_date = END_DATE)

In [ ]:
daily_means.head()

In [ ]:
# calculate mean for a category (column)
str(daily_means['Niederschlag (mm)'].mean())

In [ ]:
# calculate median for a category (column)
str(daily_means['Niederschlag (mm)'].median())

### Use Case: über mehrere Patient:innen loopen und einen Datensatz für alle

In [ ]:
# INPUT from the User Interface 
START_DATE = input("Bitte geben Sie den Startzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD). Es sind Daten ab dem 01.01.2010 verfügbar:  ")
END_DATE = input("Bitte geben Sie den Endzeitpunkt der Datenabfrage ein (Format: YYYY-MM-DD): ")

In [ ]:
# Load Patient Data - test data
with open("data/raw/2024-12-02_mii_testdaten_large.json", 'r') as file:
    data = json.load(file)

In [ ]:
# Extract Patients from Data
patients = PatientsDownload()
patients.extract_patients(data = data)

# Get Weather Data
weather_data = WeatherdataDownloader()
daily_means = weather_data.get_weatherdata_patient_collection(patients.patients, start_date = START_DATE, end_date = END_DATE)

In [ ]:
daily_means.head()

In [ ]:
# simple table to show how many patients are in which standort
daily_means_deduplicated = daily_means.drop_duplicates(subset=['standort', 'patient_id'])
grouped = daily_means_deduplicated.groupby('standort')['patient_id'].count().reset_index()
grouped.columns = ['standort', 'num_patients']

print(grouped)